In [9]:
import os
import pandas as pd
import geopandas as gpd
from shapely.ops import linemerge
from shapely.wkt import loads

%matplotlib inline

In [2]:
out_file_json = 'data/4A_1_RouteLinks.geojson'
out_file_csv = 'data/4A_1_RouteLinks.csv'

In [3]:
def flatten(geom):
    if geom.type == 'MultiLineString':
        return linemerge(geom)
    elif geom.type == 'LineString':
        return geom
    else:
        return None

In [7]:
link_shape = gpd.read_file('data/routelinks/RouteLinks.shp')
link_data = pd.read_csv('data/4A_1_Links.csv', sep = ';')

In [8]:
link_shape['PointLinkRef'] = link_shape['TripPointF'] + ':' + link_shape['TripPointT']

In [9]:
link = link_shape.merge(link_data, on = 'PointLinkRef')

In [10]:
stoplinks = link[['StopPointLinkRef', 'geometry']].dissolve(by='StopPointLinkRef').reset_index().rename(columns = {'StopPointLinkRef': 'LinkRef'})

In [11]:
stoplinks_wgs84 = stoplinks.to_crs(epsg=4326) 

In [12]:
stoplinks_wgs84['geometry'] = stoplinks_wgs84['geometry'].apply(flatten)

In [13]:
if os.path.exists(out_file_json):
    os.remove(out_file_json)
    
stoplinks_wgs84.to_file(out_file_json,driver='GeoJSON')

In [14]:
stoplinks_wgs84.to_csv(out_file_csv, index = False)

### Points to GeoJSON

In [12]:
point_data = pd.read_csv('../data/4A_1_Points.csv', sep = ';')
crs = {'init': 'epsg:4326'}
point_data = gpd.GeoDataFrame(point_data.drop(['Geography'], axis = 1), crs=crs, geometry=point_data['Geography'].apply(loads))

In [16]:
out_points_json = '../data/4A_1_Points.geojson'
point_data.to_file(out_points_json, driver='GeoJSON')